<a href="https://colab.research.google.com/github/guilherme-argentino/fiap-ia4devs-techchallenge-fase4/blob/main/Atividade4Corrigida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!apt-get update && apt-get install -y libgl1-mesa-glx
!pip install opencv-python matplotlib deepface mediapipe

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [6]:
import cv2
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from google.colab.patches import cv2_imshow
from deepface import DeepFace
import mediapipe as mp
import numpy as np


def detect_faces_and_activities(video_path, output_path, duration_seconds=None):
    """Detecta rostos, emoções e atividades no vídeo."""

    # Modelos do MediaPipe para detecção de atividades
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()

    # Modelo de detecção de rosto
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Abrindo o vídeo
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Erro: Não foi possível abrir o vídeo.")
        return

    # Obtendo propriedades do vídeo
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Definir limite de quadros para teste rápido
    if duration_seconds:
        max_frames = min(total_frames, int(fps * duration_seconds))
    else:
        max_frames = total_frames

    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0  # Contador de quadros processados

    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Converter para escala de cinza (para detecção de rosto)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detectar rostos
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Processar cada rosto detectado
        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]  # Extrair a região do rosto

            try:
                # Analisar emoção usando DeepFace
                analysis = DeepFace.analyze(face, actions=['emotion'], enforce_detection=False)
                emotion = analysis[0]['dominant_emotion']  # Obter emoção dominante

                # Desenhar retângulo no rosto
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

                # Adicionar a emoção detectada no vídeo
                cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Erro na análise de emoção: {e}")

        # **Detecção de atividade humana (pose estimation)**
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(frame_rgb)

        if result.pose_landmarks:
            mp_drawing = mp.solutions.drawing_utils
            mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Classificação simples de atividades baseada em posições dos ombros e joelhos
            landmarks = result.pose_landmarks.landmark
            if landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y < landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y:
                activity = "Em Pé"
            else:
                activity = "Sentado/Agachado"

            # Adicionar texto de atividade detectada no vídeo
            cv2.putText(frame, activity, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        # Escrever o quadro no vídeo de saída
        out.write(frame)

        # Exibir o quadro no Google Colab
        cv2_imshow(frame)
        clear_output(wait=True)

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Vídeo processado salvo como {output_path}")

# **Exemplo de uso:**
detect_faces_and_activities(
    "/content/Unlocking Facial Recognition_ Diverse Activities Analysis.mp4",
    "/content/output_video.mp4",
    duration_seconds=10  # Teste rápido com apenas 10 segundos de vídeo
)


Vídeo processado salvo como /content/output_video.mp4
